In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dashboard components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64, os

# Data / plotting
import pandas as pd

# CRUD module
from AnimalShelter import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = "aacuser"
password = "SNHU1234"

db = AnimalShelter(username, password)
df = pd.DataFrame.from_records(db.read({}))
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)

# Defensive columns so charts don’t crash if missing
for col in ["animal_type", "breed", "name"]:
    if col not in df.columns:
        df[col] = ""

#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Optional logo
image_filename = "my-image.png"
logo = None
if os.path.exists(image_filename):
    logo = "data:image/png;base64," + base64.b64encode(open(image_filename, "rb").read()).decode()

animal_types = ["All"] + sorted([x for x in df["animal_type"].dropna().unique() if x != ""])

app.layout = html.Div([
    html.Center(html.B(html.H1("CS-340 Dashboard — Lynneska Rivera"))),
    html.Hr(),
    html.Div(children=[] if not logo else [html.Img(src=logo, style={"height": "60px"})],
             style={"textAlign": "center", "marginBottom": "8px"}),

    # >>> filter control so 'filter-type' exists <<<
    html.Div([
        html.Label("Filter by Animal Type:"),
        dcc.RadioItems(
            id="filter-type",
            options=[{"label": t, "value": t} for t in animal_types],
            value="All",
            inline=True
        )
    ], style={"marginBottom": "8px"}),

    html.Hr(),

    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        sort_action="native",
        filter_action="native",
        row_selectable="single",
        selected_rows=[0],
        style_table={"overflowX": "auto"},
        style_header={"fontWeight": "bold"}
    ),

    html.Br(),
    html.Hr(),

    html.Div(className='row', style={'display': 'flex', 'gap': '16px'}, children=[
        html.Div(id='graph-id', className='col s12 m6', style={'flex': '1'}),
        html.Div(id='map-id', className='col s12 m6', style={'flex': '1'})
    ])
])

#############################################
# Interaction Between Components / Controller
#############################################

# 1) Filter the table
@app.callback(
    Output('datatable-id', 'data'),
    Input('filter-type', 'value')
)
def update_dashboard(filter_type):
    if not filter_type or filter_type == "All":
        dff = df.copy()
    else:
        dff = pd.DataFrame.from_records(db.read({"animal_type": filter_type}))
        if "_id" in dff.columns:
            dff.drop(columns=["_id"], inplace=True)
        # keep defensive columns
        for col in ["animal_type", "breed", "name"]:
            if col not in dff.columns:
                dff[col] = ""
    return dff.to_dict('records')

# 2) Chart from table view  <<< THIS WAS EMPTY BEFORE
@app.callback(
    Output('graph-id', "children"),
    Input('datatable-id', "derived_virtual_data")
)
def update_graphs(viewData):
    if not viewData:
        # fall back to full df
        data = df
    else:
        data = pd.DataFrame(viewData)
    if "breed" not in data.columns:
        return html.Div("No 'breed' column available for chart.")
    counts = data["breed"].fillna("Unknown").value_counts().reset_index()
    counts.columns = ["breed", "count"]
    fig = px.pie(counts, names="breed", values="count", title="Breed Distribution")
    return dcc.Graph(figure=fig)

# 3) Style selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    Input('datatable-id', 'selected_columns')
)
def update_styles(selected_columns):
    selected_columns = selected_columns or []
    return [{'if': {'column_id': i}, 'background_color': '#D2F3FF'} for i in selected_columns]

# 4) Map from selected row
@app.callback(
    Output('map-id', "children"),
    Input('datatable-id', "derived_virtual_data"),
    Input('datatable-id', "derived_virtual_selected_rows")
)
def update_map(viewData, index):
    if not viewData:
        return html.Div("No data to map.")
    if not index:
        index = [0]
    dff = pd.DataFrame.from_dict(viewData)
    try:
        row = index[0]
        lat = float(dff.iloc[row, 13])
        lon = float(dff.iloc[row, 14])
        breed = str(dff.iloc[row, 4])
        name = str(dff.iloc[row, 9])
    except Exception:
        # Graceful fallback (Austin, TX)
        lat, lon, breed, name = 30.75, -97.48, "Unknown", "Unknown"

    return [
        dl.Map(style={'width': '100%', 'height': '500px'}, center=[lat, lon], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([html.H4("Animal"), html.P(f"Name: {name}"), html.P(f"Breed: {breed}")])
            ])
        ])
    ]

app.run_server(debug=True, port=8050)


ERROR:root:Read failed: localhost:34195: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms), Timeout: 30s, Topology Description: <TopologyDescription id: 690fdd888d25060ed93e6ba5, topology_type: Unknown, servers: [<ServerDescription ('localhost', 34195) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:34195: [WinError 10061] No connection could be made because the target machine actively refused it (configured timeouts: socketTimeoutMS: 20000.0ms, connectTimeoutMS: 20000.0ms)')>]>
C:\Users\Admin\AppData\Local\Programs\Python\Python313\Lib\site-packages\dash\dash.py:634: UserWarning:

JupyterDash is deprecated, use Dash instead.
See https://dash.plotly.com/dash-in-jupyter for more details.



AttributeError: 'super' object has no attribute 'run_server'